In [2]:
%matplotlib widget

Trying to replicate the following research piece:
https://www.federalreserve.gov/econres/notes/feds-notes/index-of-common-inflation-expectations-20200902.htm
Essentially to create an indicator of inflation expectations that reads through the noise of various forward looking metrics.

In [4]:
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
from fredapi import Fred as FredApi

fred = FredApi(api_key='0dfcbb403eea7fee006f01e935be9216')

#### University of Michigan data
Probably the easiest solution is to download data in csv format from the university of Michigan data area located at:
https://data.sca.isr.umich.edu/data-archive/mine.php
selecting tables related to "expected changes in prices in the next..." and read from the CSVs.
You can also get selected data (not all) directly from the Fred archive.
Once you've got those, you have 4 of the 21 series proposed by the article as they use:
* short term mean
* medium term mean
* medium term 25 and 75 percentiles
But I am going to use 6 of these, same across the 2 horizons.

In [10]:
# mich_short_fred = fred.get_series_latest_release('MICH')
mich_short = pd.read_csv("michigan_inflation_short_term.csv", header=1, skiprows=0)
mich_medium = pd.read_csv("michigan_inflation_medium_term.csv", header=1, skiprows=0)

In [11]:
mich_short.head(2)

,Month,Year,Down,Same,Up by 1-2%,Up by 3-4%,Up by 5%,Up by 6-9%,Up by 10-14%,Up by 15%+,Up; DK how much,DK; NA,Mean,Variance,Standard Deviation,25th Percentile,Median,75th Percentile,Interquartile Range (75th-25th),Unnamed: 19
0,1,1978,2,13,8,11,15,17,10,3,21,0,6.1,31,5.6,3.0,5.2,7.8,4.8,NaN
1,2,1978,1,11,6,9,14,16,11,10,19,3,8.5,72,8.5,4.1,6.4,10.0,5.9,NaN


In [12]:
mich_short.tail(2)

,Month,Year,Down,Same,Up by 1-2%,Up by 3-4%,Up by 5%,Up by 6-9%,Up by 10-14%,Up by 15%+,Up; DK how much,DK; NA,Mean,Variance,Standard Deviation,25th Percentile,Median,75th Percentile,Interquartile Range (75th-25th),Unnamed: 19
521,6,2021,3,6,15,23,15,6,10,12,10,0,6.1,44,6.6,2.4,4.2,9.5,7.1,NaN
522,7,2021,2,10,11,18,18,11,12,9,8,1,5.8,32,5.7,2.5,4.7,7.9,5.4,NaN


Now that you have checked where the stuff comes from and what the dates are, you can create an index in pandas and assign it.

In [13]:
mich_short['Day'] = 1
mich_medium['Day'] = 1
mich_short['Date'] = pd.to_datetime(mich_short[['Year','Month','Day']])
mich_medium['Date'] = pd.to_datetime(mich_medium[['Year','Month','Day']])
mich_short.set_index('Date', inplace=True)
mich_medium.set_index('Date', inplace=True)
mich_short = mich_short.drop(['Year','Month','Day','Unnamed: 19'], axis=1)
mich_medium = mich_medium.drop(['Year','Month','Day','Unnamed: 19'], axis=1)

In [14]:
mich_short.tail()

,Down,Same,Up by 1-2%,Up by 3-4%,Up by 5%,Up by 6-9%,Up by 10-14%,Up by 15%+,Up; DK how much,DK; NA,Mean,Variance,Standard Deviation,25th Percentile,Median,75th Percentile,Interquartile Range (75th-25th)
Date,,,,,,,,,,,,,,,,,
2021-03-01,2,12,22,18,12,6,9,5,13,1,4.3,18,4.3,1.3,3.1,5.5,4.2
2021-04-01,3,9,19,20,16,7,9,4,12,1,4.3,16,4.0,1.6,3.4,5.4,3.8
2021-05-01,2,6,14,21,17,7,11,9,13,0,5.7,30,5.5,2.6,4.6,7.5,5.0
2021-06-01,3,6,15,23,15,6,10,12,10,0,6.1,44,6.6,2.4,4.2,9.5,7.1
2021-07-01,2,10,11,18,18,11,12,9,8,1,5.8,32,5.7,2.5,4.7,7.9,5.4


In [15]:
mich_medium.tail()

,Down,Same,Up by 1-2%,Up by 3-4%,Up by 5%,Up by 6-9%,Up by 10-14%,Up by 15%+,Up; DK how much,DK; NA,Mean,Variance,Standard deviation,25th Percentile,Median,75th Percentile,Interquartile Range (75th-25th)
Date,,,,,,,,,,,,,,,,,
2021-03-01,2.0,3.0,32.0,26.0,11.0,4.0,7.0,1.0,12.0,2.0,3.4,9.0,3.1,1.5,2.8,4.6,3.1
2021-04-01,2.0,2.0,36.0,25.0,15.0,4.0,2.0,1.0,12.0,1.0,3.0,6.0,2.5,1.4,2.7,4.5,3.1
2021-05-01,5.0,2.0,27.0,27.0,13.0,5.0,6.0,1.0,12.0,2.0,3.5,10.0,3.2,1.7,3.0,4.8,3.1
2021-06-01,8.0,2.0,29.0,26.0,11.0,4.0,7.0,2.0,8.0,3.0,3.4,16.0,4.0,1.5,2.8,4.7,3.2
2021-07-01,6.0,5.0,28.0,24.0,11.0,7.0,5.0,3.0,8.0,3.0,3.5,15.0,3.9,1.4,2.8,4.8,3.4


In [16]:
plt.figure()
ax = mich_short['Mean'].plot();
mich_medium['Mean'].plot();
ax.grid(True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
# now construct the core DataFrame of the Michigan data and store it in a pickle
columns = ["mich_short_mean","mich_short_25","mich_short_75","mich_medium_mean","mich_medium_25","mich_medium_75"]
michigan_dataset = pd.DataFrame(columns=columns, index=mich_short.index)
michigan_dataset[["mich_short_mean","mich_short_25","mich_short_75"]] = mich_short[["Mean","25th Percentile","75th Percentile"]].to_numpy()
michigan_dataset[["mich_medium_mean","mich_medium_25","mich_medium_75"]] = mich_medium[["Mean","25th Percentile","75th Percentile"]].to_numpy()
michigan_dataset = michigan_dataset.resample('M').last().dropna()
michigan_dataset.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 418 entries, 1979-02-28 to 2021-07-31
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   mich_short_mean   418 non-null    float64
 1   mich_short_25     418 non-null    float64
 2   mich_short_75     418 non-null    float64
 3   mich_medium_mean  418 non-null    float64
 4   mich_medium_25    418 non-null    float64
 5   mich_medium_75    418 non-null    float64
dtypes: float64(6)
memory usage: 22.9 KB


In [18]:
michigan_dataset.tail()

,mich_short_mean,mich_short_25,mich_short_75,mich_medium_mean,mich_medium_25,mich_medium_75
Date,,,,,,
2021-03-31,4.3,1.3,5.5,3.4,1.5,4.6
2021-04-30,4.3,1.6,5.4,3.0,1.4,4.5
2021-05-31,5.7,2.6,7.5,3.5,1.7,4.8
2021-06-30,6.1,2.4,9.5,3.4,1.5,4.7
2021-07-31,5.8,2.5,7.9,3.5,1.4,4.8


In [19]:
michigan_dataset.to_pickle('michigan_dataset.pd.pkl')